In [29]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans


In [30]:
def load_train_data(fileName):
    w = np.genfromtxt(fileName, delimiter=',')
    x_train = w[1:,1:-1]
    y_train = w[1:,-1].astype(int)
    return x_train, y_train
    
def load_test_data(fileName):
    w = np.genfromtxt(fileName, delimiter=',')
    x_test = w[1:,1:]
    return x_test

In [31]:
def marg(prob,Y_test):
    M = np.array([(j[k] - np.max(np.append(j[:k],j[k+1:])))  for j,k in zip(prob,Y_test)])
    ind = np.where(M>0.75)
    M = np.array(sorted(M))
    c = list(range(len(M)))
    plt.plot(c,M, )
    plt.hlines(0,0,len(c),color='black')
    plt.fill_between(c, M,where = M<0,color='red',alpha=0.5)
    plt.fill_between(c, M,where = M>0,color='green',alpha=0.5)


In [32]:

def marg_et(prob,Y_test,X_test):
    print(Y_test)
    M = np.array([(j[k] - np.max(np.append(j[:k],j[k+1:])))  for j,k in zip(prob,Y_test)])
    ind = np.where(M>0.5)
    etalon_x  = np.copy(X_test[ind])
    etalon_y  = np.copy(Y_test[ind])
    M = np.array(sorted(M))
    c = list(range(len(M)))
    plt.plot(c,M, )
    plt.hlines(0,0,len(c),color='black')
    plt.fill_between(c, M,where = M<0,color='red',alpha=0.5)
    plt.fill_between(c, M,where = M>0,color='green',alpha=0.5)
    return [etalon_x,etalon_y]

In [33]:
def clusters(x_train, y_train):
    yy = np.array([],dtype=np.int64)
    for i in range(10):
        
        count = 1000#int(np.unique(y_train, return_counts=True)[1][i]//1.5)
        #print(count)
        ind_i = np.where(y_train == i)[0]
        kmeans = KMeans(n_clusters=count, random_state=0).fit(x_train[ind_i])
        if i == 0:
            list_clusters = kmeans.cluster_centers_
        else:
            list_clusters = np.vstack((list_clusters, kmeans.cluster_centers_))
        yy =  np.append(yy, np.array([i]*count))
        int_array = yy.astype(int)
    return list_clusters, int_array
    

In [34]:
x_train, y_train = load_train_data('train.csv')

In [35]:
x_test = load_test_data('test.csv')

In [36]:
col_mean = np.nanmean(x_test, axis=0)
inds = np.where(np.isnan(x_test))

#Place column means in the indices. Align the arrays using take
x_test[inds] = np.take(col_mean, inds[1])


In [37]:
col_mean = np.nanmean(x_train, axis=0)
inds = np.where(np.isnan(x_train))

#Place column means in the indices. Align the arrays using take
x_train[inds] = np.take(col_mean, inds[1])


In [38]:

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0) 
print('X_train_norm.shape, y_train.shape: ', x_train.shape, y_train.shape)
print('X_train.shape, Y_train.shape, X_test.shape, Y_test.shape: ', X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

X_train_norm.shape, y_train.shape:  (50000, 110) (50000,)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape:  (40000, 110) (40000,) (10000, 110) (10000,)


In [39]:
kf = KFold(n_splits=3)
kf.get_n_splits(X_train)

3

In [ ]:
X_train.shape

## Logistic Regression.Один против всех

In [ ]:
Accuracy = 0
for train_index, test_index in kf.split(X_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_v, X_test_v = X_train[train_index], X_train[test_index]
    Y_train_v, Y_test_v = Y_train[train_index], Y_train[test_index]
    
    
    scaler = StandardScaler()
    scaler.fit(X_train_v)
    X_train_v = scaler.transform(X_train_v)
    X_test_v = scaler.transform(X_test_v)
    
    X_train_v, Y_train_v = clusters(X_train_v,Y_train_v)
    #print(X_train_v)
    models = []
    for i in range(0,10):
        model = LogisticRegression(random_state=0,max_iter=200,solver = 'sag')
        yy = np.array(Y_train_v==i,dtype = np.int64)
        model.fit(X_train_v,yy)
        models.append(model)
    y_preds = []
    for i in range(0,10):
        y_pred = models[i].predict_proba(X_test_v)
        y_preds.append(y_pred)
    y_preds = np.array(y_preds)
    res = np.hstack((y_preds[0:10,:,-1:][0],y_preds[0:10,:,-1:][1]))
    for i in range(2,10):
        res = np.hstack((res,y_preds[0:10,:,-1:][i]))
    marg(res,Y_test_v)
    amswer = np.argmax(res, axis=1)
    #print('Accuracy',np.sum(Y_test_v == amswer)/len(Y_test_v))
    Accuracy+=np.sum(Y_test_v == amswer)/len(Y_test_v)
print('Accuracy',Accuracy/kf.get_n_splits(X_train))

In [56]:
Accuracy = 0
for train_index, test_index in kf.split(X_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_v, X_test_v = X_train[train_index], X_train[test_index]
    Y_train_v, Y_test_v = Y_train[train_index], Y_train[test_index]
    
    clusters

    #scaler = StandardScaler()
    #scaler.fit(X_train_v)
    #X_train_v = scaler.transform(X_train_v)
    #X_test_v = scaler.transform(X_test_v)
    #print(X_train_v)
    
    model = LogisticRegression(solver="sag", max_iter=1000, random_state=42, multi_class='multinomial')
    model.fit(X_train_v,Y_train_v)
    amswer = model.predict(X_test_v)
    #marg(res,Y_test_v)
    #amswer = np.argmax(res, axis=1)
    #print('Accuracy',np.sum(Y_test_v == amswer)/len(Y_test_v))
    Accuracy+=np.sum(Y_test_v == amswer)/len(Y_test_v)
    print(Accuracy)
print('Accuracy',Accuracy/kf.get_n_splits(X_train))

0.6789410529473526
1.3580080296217694
2.0386500456721706
Accuracy 0.6795500152240569


In [ ]:
Accuracy = 0
for train_index, test_index in kf.split(X_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_v, X_test_v = X_train[train_index], X_train[test_index]
    Y_train_v, Y_test_v = Y_train[train_index], Y_train[test_index]
    
    
    scaler = StandardScaler()
    scaler.fit(X_train_v)
    X_train_v = scaler.transform(X_train_v)
    X_test_v = scaler.transform(X_test_v)
    
    X_train_v, Y_train_v = clusters(X_train_v,Y_train_v)
    #print(X_train_v)
    models = []
    for i in range(0,10):
        model =  Perceptron( random_state=42)
        yy = np.array(Y_train_v==i,dtype = np.int64)
        model.fit(X_train_v,yy)
        models.append(model)
    y_preds = []
    for i in range(0,10):
        y_pred = models[i].predict_proba(X_test_v)
        y_preds.append(y_pred)
    y_preds = np.array(y_preds)
    res = np.hstack((y_preds[0:10,:,-1:][0],y_preds[0:10,:,-1:][1]))
    for i in range(2,10):
        res = np.hstack((res,y_preds[0:10,:,-1:][i]))
    marg(res,Y_test_v)
    amswer = np.argmax(res, axis=1)
    #print('Accuracy',np.sum(Y_test_v == amswer)/len(Y_test_v))
    Accuracy+=np.sum(Y_test_v == amswer)/len(Y_test_v)
print('Accuracy',Accuracy/kf.get_n_splits(X_train))

In [55]:
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC



Accuracy = 0
for train_index, test_index in kf.split(X_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_v, X_test_v = X_train[train_index], X_train[test_index]
    Y_train_v, Y_test_v = Y_train[train_index], Y_train[test_index]
    

    #print(X_train_v)
    
    model_SVC = SVC(random_state = 42, C=1.0, kernel ='poly',gamma='scale')
    model_KNN = KNeighborsClassifier(n_neighbors=25, weights=fun)
    model_LR = LogisticRegression(solver="sag", max_iter=1000, random_state=42, multi_class='multinomial')

    
    model_SVC.fit(X_train_v,Y_train_v)
    model_KNN.fit(X_train_v,Y_train_v)
    model_LR.fit(X_train_v,Y_train_v)
    
    amswer_svc = model_SVC.predict(X_test_v)
    amswer_knn = model_KNN.predict(X_test_v)
    amswer_lr = model_LR.predict(X_test_v)
    
    amswer = np.vstack((np.vstack((amswer_svc,amswer_knn)),amswer_lr))
    amswer = np.array(mostCommon(amswer))
    
    Accuracy+=np.sum(Y_test_v == amswer)/len(Y_test_v)
    print(Accuracy)
print('Accuracy',Accuracy/kf.get_n_splits(X_train))


0.7692365381730913
1.5380057574035721
2.3066249728839594
Accuracy 0.7688749909613198


In [46]:
x = np.array([1,2,3])
y = np.array([1,1,3])
z = np.array([3,2,3])

In [47]:
res = np.vstack((np.vstack((x,y)),z))

In [51]:
def mostCommon(lst):
    return [Counter(col).most_common(1)[0][0] for col in zip(*lst)]

In [53]:
np.array(mostCommon(res))

array([1, 2, 3])

In [50]:
b

Counter({3: 3})

In [27]:
res = np.max(res, axis=0)

In [28]:
res

array([3, 2, 3])

## KNN

In [43]:
def fun(dist):
    ret = np.ones_like(dist)
    k = ret.shape[1]
    for i in range(k):
        ret[:,i] *=(k-i)/k
    return ret
    

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_t = scaler.transform(X_train)
X_test_t = scaler.transform(X_test)
    #print(X_train_v)

model =  SVC(random_state = 42, C=1.0, kernel ='poly',gamma='scale')
model.fit(X_train_t,Y_train)
res = model.predict(X_test_t)
Accuracy=np.sum(Y_test == res)/len(Y_test)



In [ ]:
print(Accuracy)

In [ ]:
c = 1 0.7777

In [ ]:
model =  KNeighborsClassifier(n_neighbors=25, weights=fun)
model.fit(x_train, y_train)
res = model.predict(x_test)
#print(np.sum(Y_test == y_pred)/len(Y_test))
col = np.array(range(0,10000))
result = np.c_[col,res]
print(result)
with open("sample_sabmission.csv", "wb") as f:
    f.write(b'id,label\n')
    np.savetxt(f, result.astype(int), fmt='%i', delimiter=",")